# Exercise 2: Spark Fundamentals

## Learning Objectives
- Understand the difference between RDDs and DataFrames
- Master transformations (lazy) vs actions (eager)
- Create and manipulate distributed datasets
- View job execution in the Spark UI

---

## Part 1: Creating a SparkSession

In [ ]:
from pyspark.sql import SparkSession

# Create SparkSession connected to YARN
spark = SparkSession.builder \
    .appName("Spark Fundamentals Lab") \
    .master("yarn") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Application ID: {spark.sparkContext.applicationId}")

🔗 **Open Spark UI**: http://localhost:4040 to monitor your application

## Part 2: Working with RDDs (Resilient Distributed Datasets)

RDDs are the foundational data structure in Spark - immutable, distributed collections.

In [ ]:
# Create an RDD from a Python list
numbers = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], numSlices=4)
print(f"Number of partitions: {numbers.getNumPartitions()}")

In [ ]:
# Transformations are LAZY - they don't execute immediately
squared = numbers.map(lambda x: x ** 2)
filtered = squared.filter(lambda x: x > 20)

print("Transformations defined, but nothing executed yet!")
print(f"Type: {type(filtered)}")

In [ ]:
# Actions TRIGGER execution
result = filtered.collect()
print(f"Result: {result}")

### 🔍 Checkpoint Question 2
Check the Spark UI (Jobs tab). How many jobs were triggered by the code above?
Which operation triggered the job - `map()`, `filter()`, or `collect()`?

**Your Answer:**

## Part 3: Common RDD Transformations and Actions

In [ ]:
# Read file as RDD
lines_rdd = spark.sparkContext.textFile("hdfs:///user/student/data/transactions.csv")

# Skip header and parse
header = lines_rdd.first()
data_rdd = lines_rdd.filter(lambda line: line != header)

In [ ]:
# Transformations
def parse_transaction(line):
    fields = line.split(',')
    return {
        'transaction_id': fields[0],
        'amount': float(fields[7]) if len(fields) > 7 else 0.0
    }

parsed = data_rdd.map(parse_transaction)
amounts = parsed.map(lambda x: x['amount'])

In [ ]:
# Actions
print(f"Count: {amounts.count()}")
print(f"Sum: ${amounts.sum():,.2f}")
print(f"Average: ${amounts.mean():,.2f}")
print(f"Max: ${amounts.max():,.2f}")

## Part 4: DataFrames - The Modern API

DataFrames are structured, schema-aware, and optimized with Catalyst.

In [ ]:
# Read CSV with schema inference
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs:///user/student/data/transactions.csv")

df.printSchema()

In [ ]:
# Show sample data
df.show(5)

In [ ]:
# DataFrame transformations
from pyspark.sql.functions import col, sum, avg, count

# Group by region and calculate totals
region_summary = df.groupBy("store_region") \
    .agg(
        count("*").alias("transaction_count"),
        sum("total_amount").alias("total_sales"),
        avg("total_amount").alias("avg_transaction")
    ) \
    .orderBy("total_sales", ascending=False)

region_summary.show()

## Part 5: Understanding Lazy Evaluation

In [ ]:
# These are all transformations - nothing executes yet!
step1 = df.filter(col("is_online") == True)
step2 = step1.select("transaction_id", "total_amount", "payment_method")
step3 = step2.filter(col("total_amount") > 100)

print("Built the execution plan. Check Spark UI - no new jobs yet!")

In [ ]:
# View the execution plan
step3.explain(True)

In [ ]:
# Trigger execution with an action
result_count = step3.count()
print(f"High-value online transactions: {result_count}")

## Cleanup

In [ ]:
# Stop SparkSession when done
spark.stop()
print("SparkSession stopped.")